In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from llama_cpp import Llama

# Crear instancia de FastAPI
app = FastAPI()

# Definir esquema del request
class PromptRequest(BaseModel):
    prompt: str
    max_tokens: int = 100

# Cargar el modelo
llm = Llama(model_path="./qwen2.5.gguf", n_ctx=512)

# Ruta para generar texto
@app.post("/generate")
async def generate_text(req: PromptRequest):
    output = llm(req.prompt, max_tokens=req.max_tokens)
    return {"response": output["choices"][0]["text"]}

llama_model_load_from_file_impl: using device Metal (Apple M1 Max) - 21839 MiB free
llama_model_loader: loaded meta data with 26 key-value pairs and 435 tensors from ./qwen2.5.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Qwen2.5 Coder 3B Instruct AWQ
llama_model_loader: - kv   3:                           general.finetune str              = Instruct-AWQ
llama_model_loader: - kv   4:                           general.basename str              = Qwen2.5-Coder
llama_model_loader: - kv   5:                         general.size_label str              = 3B
llama_model_loader: - kv   6:                          qwen2.block_cou

In [1]:
from llama_cpp import Llama

llm = Llama(model_path="./qwen2.5.gguf", n_ctx=512)

ModuleNotFoundError: No module named 'llama_cpp'

In [4]:
pip install llama-cpp-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 20.0 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.9-cp310-cp310-macosx_15_0_arm64.whl size=3520875 sha256=654ddcf8bca9863406b4ce61693914887dedca7179ca1b7d913518166ca56126
  Stored in directory: /Users/sebastianulloa/Library/Caches/pip/wheels/e2/91/0a/79c7b44fab10c7222ec91bd97fd7f6708beba84d5934228a80
Successfully built llama-cpp-python

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
import psycopg2

def ejecutar_sql(query: str):
    conn = psycopg2.connect(
        dbname="postgres",
        user="sebastianulloa",
        password="tu_tugimalo1contraseña",
        host="localhost",
        port="5432"
    )
    cur = conn.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    columns = [desc[0] for desc in cur.description]
    cur.close()
    conn.close()
    
    # Arma un texto tipo tabla
    resultados = [dict(zip(columns, row)) for row in rows]
    return resultados

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from llama_cpp import Llama
from typing import List
import json

app = FastAPI()

class PromptRequest(BaseModel):
    prompt: str
    max_tokens: int = 100

llm = Llama(model_path="./qwen2.5.gguf", n_ctx=1024)

@app.post("/chat")
async def chat_con_datos(req: PromptRequest):
    # Paso 1: traducir pregunta a consulta SQL (a mano o via prompt)
    if "clientes" in req.prompt.lower():
        query = "SELECT first_name, rut_number FROM beneficiaries LIMIT 5;"
    else:
        return {"response": "No entendí qué buscar. Pregunta sobre 'clientes' o 'ventas'."}

    # Paso 2: ejecutar la consulta
    datos = ejecutar_sql(query)
    contexto = json.dumps(datos, indent=2)

    # Paso 3: crear prompt para el LLM
    prompt = f"""Usa la siguiente información para responder:
    
    {contexto}
    
    Pregunta: {req.prompt}
    Respuesta:"""

    output = llm(prompt, max_tokens=req.max_tokens)
    return {"response": output["choices"][0]["text"]}